In [24]:
# Program Purpose: Find Correlation between target column and all other columns. Automates statistical 
# tests of correlation as well as allows any analyst to find association between all types of variables 
# without extensive statistical experience.

# First, the program takes an input of a excel file or a csv file and
# then have the user input a target column. This target will then be compared to all of the 
# other columns through the use of statistical tests. These tests will then determine correlation
# between the target and the columns. The program classifies each of these correlations as
# either large, medium, small, or none. The program outputs a list of all of these correlations
# based on the category that they were assigned. 


import pandas as pd
import numpy as np
from scipy import stats

df = pd.read_excel('C:/Users/Ted/Downloads/FinancialSample.xlsx')
#df = pd.read_csv('C:/Users/Ted/Downloads/FinancialSample.csv')

target = 'Segment'         #Segment and Units Sold are good test samples
showCrosstab = False
showGroupMeans = False        # Not working for now due to the possibility of the order of object and number to be swapped in ANOVA
includeExplanations = True    # Not implemented

correlation_categories = {
    "large": [],
    "medium": [],
    "small": [],
    "none": []
}

# for every column that is not the target column, check the dtype of that 
# column and the target column and see if they are both object
for column in df.columns:
    if target != column:
        #object vs object: chi2
        # this test compares observed and expected frequencies of the crosstab
        if df[target].dtype == 'object' and df[column].dtype == 'object':
            ct = pd.crosstab(df[target], df[column])
            if showCrosstab == True:
                print(ct)
                print()
            
            chi2, p, dof, expected = stats.chi2_contingency(ct)
            critical_value = stats.chi2.ppf(1 - 0.05, dof)
            
                    
            if p < 0.05:
                if chi2 > critical_value:
                    
                    print(f'Target: {target}')
                    print(f'Column: {column}')
                    print()
                    
                    n = np.sum(ct.values) 
                    cramers_v = np.sqrt(chi2 / (n * (min(ct.shape) - 1)))
    
                    if cramers_v > 0.5:
                        doc = 'Large'
                        correlation_categories["large"].append(column)
                    elif cramers_v > 0.3:
                        doc = 'Medium'
                        correlation_categories["medium"].append(column)
                    elif cramers_v > 0.1:
                        doc = 'Small'
                        correlation_categories["small"].append(column)
                    else:
                        doc = 'None'
                        correlation_categories["none"].append(column)

                    print(f'chi2: {chi2}')
                    print(f'Critical Value: {critical_value}')          # this is the value that chi squared must be below or above to determine if it is high or low
                    print(f'p: {p}')
                    
                    print()    
                    print('Correlation:', doc)
                    print(f'Cramers V: {cramers_v}')            # note that Cramers V works but is not suited towards 2x2 contingency tables
            
                    #print('p value is below the significance threshold of 0.05')
                    #print('chi2 value is greater than critical value')
                    #print('the difference between observed and expected frequencies is too large to be attributed to chance')

                    print()
                    print('---------------------------------------------------------')
                    print()
                    
                else: 
                    correlation_categories["none"].append(column)
                    #print('No Correlation')
                    #print()
                    #print('p value is above the significance threshold of 0.05')
                    #print('chi2 value is less than critical value')
                    #print('the difference between observed and expected frequencies is too small and could be attributed to chance')
                    
            else:
                correlation_categories["none"].append(column)
                #print('No Correlation')
                #print()
                #print('p value is above the significance threshold of 0.05')
             
            

            
        #numerical vs numerical: pearsons or spearmans
        if (df[target].dtype == 'float64' or df[target].dtype == 'int64') and (df[column].dtype == 'float64' or df[column].dtype == 'int64'):
            
            x, p = stats.pearsonr(df[target], df[column])       # can use either pearsonr or spearmanr depending on how the data is distributed             
            
            if p < 0.05:
                if x >= 0.5:
                    doc = 'Large Positive'
                    correlation_categories["large"].append(column)
                elif x >= 0.3:
                    doc = 'Medium Positive'
                    correlation_categories["medium"].append(column)
                elif x >= 0.1:
                    doc = 'Small Positive'
                    correlation_categories["small"].append(column)
                elif x < 0.1 and x > -0.1:
                    doc = 'None'
                    correlation_categories["none"].append(column)
                elif x <= -0.1 and x > -0.3:
                    doc = 'Small Negative'
                    correlation_categories["small"].append(column)
                elif x <= -0.3 and x > -0.5:
                    doc = 'Medium Negative'
                    correlation_categories["medium"].append(column)
                elif x <= -0.5 and x >= -1:
                    doc = 'Large Negative'
                    correlation_categories["large"].append(column)

                print(f'Target: {target}')
                print(f'Column: {column}')
                print()

                print(f'x: {x}')
                print(f'p: {p}')

                print()
                
                print('Correlation:', doc)
                #print('p value is below the significance threshold of 0.05')    
                
                print()
                print('---------------------------------------------------------')
                print()
                
            else:
                correlation_categories["none"].append(column)
                #print('No Correlation')
                #print('Reason: p value is above the significance threshold of 0.05')
            
            
            
        #object vs numerical: ANOVA
        #There can be no correlation between nominal vs numerical, only something similar 
        if (df[target].dtype == 'object' and (df[column].dtype == 'float64' or df[column].dtype == 'int64')) or ((df[target].dtype == 'float64' or df[target].dtype == 'int64') and df[column].dtype == 'object'):    
            
            if df[target].dtype == 'object':
                grouped_data = df.groupby(target)[column].apply(list)
            else:
                grouped_data = df.groupby(column)[target].apply(list)

            # Convert the grouped data into a list of arrays suitable for ANOVA
            data_arrays = [group for group in grouped_data]

            #print(*data_arrays)
            #print(grouped_data)
            #print(grouped_data)
                
            # Perform ANOVA
            f, p = stats.f_oneway(*data_arrays)
            n = len(df.index)        # number of rows
            # Finding degrees of freedom for both numerator and demoninator, then using those to help find critical value
            dfn, dfd = (len(grouped_data) - 1), (n - 2)      # (2-1) because we are only comparing 2 groups, len(df.index) is number of rows
            critical_value = stats.f.ppf(1 - 0.05, dfn, dfd)
            
            
            if p < 0.05:
                if f > critical_value:
                    
                    # Calculating the 3 values here that are needed to find the eta^2
                    # Calculate Total Sum of Squares (TSS)  
                    # all_data = np.concatenate(grouped_data)
                    all_data = []
                    for group in grouped_data:
                        all_data.extend(group)
                    all_data = np.array(all_data)
                    grand_mean = np.mean(all_data)
                    tss = np.sum((all_data - grand_mean) ** 2)

                    # Calculate Within-Group Sum of Squares (WSS)
                    wss = sum(sum((group - np.mean(group)) ** 2) for group in grouped_data)

                    # Calculate Between-Group Sum of Squares (BSS)
                    bss = tss - wss

                    # Calculate Eta-squared (η²)
                    eta_squared = bss / tss
                    
                    if eta_squared > 0.14:
                        doc = 'Large'
                        correlation_categories["large"].append(column)
                    elif eta_squared > 0.06:
                        doc = 'Medium'
                        correlation_categories["medium"].append(column)
                    elif eta_squared > 0.01:
                        doc = 'Small'
                        correlation_categories["small"].append(column)
                    else:
                        doc = 'None'
                        correlation_categories["none"].append(column)
                        
                    print(f'Target: {target}')
                    print(f'Column: {column}')  
                    
                    if showGroupMeans == True:
                        print()
                        print('Table of Means by Each Group')
                        print()
                        group_means = df.groupby(column)[target].mean().reset_index()
                        print(round(group_means, 2))
                    
                    print()
                    print("f:", f)
                    print(f"Critical Value: {critical_value}", )
                    print("p:", p)
                    print()
                    
                    print("Eta-squared:", eta_squared)     
                    print()
                    
                    print('Correlation:', doc)
                    
                    print()
                    #print('p value is below the significance threshold of 0.05')
                    #print('f value is greater than critical value')
                    #print('the difference between observed and expected frequencies is too large to be attributed to chance')

                    print('---------------------------------------------------------')
                    print()    
                    
                else:
                    correlation_categories["none"].append(column)
                    #print('No Correlation')
                    #print()
                    #print('p value is above the significance threshold of 0.05')
                    #print('f value is less than critical value')
                    #print('the difference between observed and expected frequencies is too small and could be attributed to chance')
                    
            else:
                correlation_categories["none"].append(column)
                #print('No Correlation')
                #print()
                #print('p value is above the significance threshold of 0.05')
                

print('Correlation Between Target and All Columns')
print()
print('Target:', target)
print()
    
# printing out the values in each category
for category, values in correlation_categories.items():
    if values:
        print(category.capitalize())
        for value in values:
            print(f" - {value}")
            
            
            

Target: Segment
Column: Discount Band

chi2: 19.200
cv: 15.507
p: 0.014

Correlation: Small
Cramers V: 0.122

---------------------------------------------------------

Target: Segment
Column: Sale Price

f: 126.895
cv: 2.385
p: 0.000

Eta-squared: 0.422

Correlation: Large

---------------------------------------------------------

Target: Segment
Column: Gross Sales

f: 69.213
cv: 2.385
p: 0.000

Eta-squared: 0.285

Correlation: Large

---------------------------------------------------------

Target: Segment
Column: Discounts

f: 44.317
cv: 2.385
p: 0.000

Eta-squared: 0.203

Correlation: Large

---------------------------------------------------------

Target: Segment
Column: Sales

f: 67.745
cv: 2.385
p: 0.000

Eta-squared: 0.281

Correlation: Large

---------------------------------------------------------

Target: Segment
Column: COGS

f: 85.769
cv: 2.385
p: 0.000

Eta-squared: 0.330

Correlation: Large

---------------------------------------------------------

Target: Segment
